In [1]:
import re
from pyspark import SparkConf, SparkContext

sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [2]:
# filter articles without word when reading file
def wordFilter(line):
    text = line.lower()
    if broadcastVars.value["word"] in text:
        return text
    return None

broadcastVars = sc.broadcast({"word": "narodnaya"})

wiki = sc.textFile("/data/wiki/en_articles_part/articles-part").map(wordFilter).filter(lambda v: v != None )

In [3]:
# extract searching words pair
def extractPairs(line):
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text.lower(), flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
        
        pair = False
        pairs = []
        
        for word in words:
            if broadcastVars.value["word"] == word:
                pair = True
                continue
            if pair:
                pairs.append(broadcastVars.value["word"] + "_" + word)
                pair = False
                
        return pairs
        
    except ValueError as e:
        print e
        return []

In [4]:
result = wiki.flatMap(extractPairs).map(lambda k: (k,1)).reduceByKey(lambda x,y: x+y).collect()

In [5]:
for pair in result:
    print pair[0], pair[1]

narodnaya_gazeta 1
narodnaya_volya 9
